In [149]:
import os
# Suppress specific TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # 3 means to filter out all INFO and WARNING logs
import warnings
# Suppress specific warnings
warnings.filterwarnings('ignore', category=UserWarning, message='.*OUT_OF_RANGE.*')

import sys
sys.path.append('../src')
import random
import re
import pickle
import pandas as pd
from lr_schedular import CustomSchedule
from transformer_encoder import TransformerEncoderV3  
from positional_encoding import encode_pos_sin_cosine
import seaborn as sns
import numpy as np
import nltk
from datasets import load_dataset
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Dropout, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
from datasets import load_dataset
# Load an example dataset, 'wikipedia' for English, 2020-03-01 version
dataset = load_dataset("wikipedia", "20220301.en", split="train")
print('how the dataset looks:', dataset[0].keys())
num_of_articles = 1000

how the dataset looks: dict_keys(['id', 'url', 'title', 'text'])


In [3]:
first_article = dataset[0]['text']
print('words in first_article:', len(first_article))
print('how the article look:\n',first_article[:1500])
print('..........................................\n........................')
print(first_article[-500:])

words in first_article: 43985
how the article look:
 Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy. Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful. As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.

Humans lived in societies without formal hierarchies long before the establishment of formal states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist thought are found throughout history, modern anarchism emerged from the Enlightenment. During the latter half of the 19th and the first decades of th

Document and Sentence Segmentation:

Document Boundary: Each Wikipedia article can be treated as a single document. This aligns with the BERT requirement where each document is separated by blank lines.

Sentence Tokenization: Use a sentence tokenizer to convert each paragraph into distinct sentences. This is crucial because BERT's NSP task assumes that two consecutive sentences in the data might be used as training pairs.

In [4]:
def save_articles_with_doc_boundary(dataset_name, config_name, split_name, output_file_path, num_of_articles=1000):
    nltk.download('punkt')
    dataset = load_dataset(dataset_name, config_name, split=split_name)
    articles_to_process = dataset.select(range(num_of_articles))
    with open(output_file_path, 'w', encoding='utf-8') as file:
        for i, article in enumerate(articles_to_process):
            title = article.get('title', f"No Title Available for Article {i}")
            art_id = article.get('id', "No ID")
            art_url = article.get('url', "No URL")
            sentences = nltk.sent_tokenize(article['text'])
            full_article_text = f"ARTICLE-{i}-{art_id}-{art_url}-{title}\n" + '\n'.join(sentences) + '\n\n'
            file.write(full_article_text)
# Call the function to process and save articles
output_file_path = 'wiki_articles_with_seperator.txt'
save_articles_with_doc_boundary('wikipedia', '20220301.en', 'train', output_file_path)

[nltk_data] Downloading package punkt to /home/bhujay/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
def display_article_lines(file_path, num_lines=5, num_articles=5):    
    with open(file_path, 'r', encoding='utf-8') as file:
        article_count = 0
        line_count = 0
        for line in file:
            if line.startswith('ARTICLE-'):  # New article detected
                if article_count >= num_articles:
                    break                          
                article_count += 1
                line_count = 0  # Reset line count for the new article
            if line_count < num_lines:
                print(line)
                line_count += 1
            else:
                continue  # Skip further lines until the next article starts
output_file_path = 'wiki_articles_with_seperator.txt'
display_article_lines(output_file_path, num_lines=5, num_articles=5)

ARTICLE-0-12-https://en.wikipedia.org/wiki/Anarchism-Anarchism

Anarchism is a political philosophy and movement that is sceptical of authority and rejects all involuntary, coercive forms of hierarchy.

Anarchism calls for the abolition of the state, which it holds to be unnecessary, undesirable, and harmful.

As a historically left-wing movement, placed on the farthest left of the political spectrum, it is usually described alongside communalism and libertarian Marxism as the libertarian wing (libertarian socialism) of the socialist movement, and has a strong historical association with anti-capitalism and socialism.

Humans lived in societies without formal hierarchies long before the establishment of formal states, realms, or empires.

ARTICLE-1-25-https://en.wikipedia.org/wiki/Autism-Autism

Autism is a neurodevelopmental disorder characterized by difficulties with social interaction and communication, and by restricted and repetitive behavior.

Parents often notice signs during th

In [6]:
class TrainingInstance:
    """A single training instance (sentence pair)."""
    def __init__(self, tokens, segment_ids, masked_lm_positions, masked_lm_labels, is_random_next):
        self.tokens = tokens
        self.segment_ids = segment_ids
        self.masked_lm_positions = masked_lm_positions
        self.masked_lm_labels = masked_lm_labels
        self.is_random_next = is_random_next

    def __str__(self):
        tokens_str = " ".join([str(token) for token in self.tokens])
        segment_ids_str = " ".join(map(str, self.segment_ids))
        masked_lm_positions_str = " ".join(map(str, self.masked_lm_positions))
        masked_lm_labels_str = " ".join([str(label) for label in self.masked_lm_labels])
        return f"Tokens: {tokens_str}\nSegment IDs: {segment_ids_str}\n" \
               f"Is Random Next: {self.is_random_next}\n" \
               f"Masked LM Positions: {masked_lm_positions_str}\n" \
               f"Masked LM Labels: {masked_lm_labels_str}\n"

    def __repr__(self):
        return self.__str__()

def mask_tokens(tokens, tokenizer, max_predictions_per_seq, rng):
    """Masks tokens and returns masked tokens and corresponding labels."""
    output_tokens = tokens[:]
    output_labels = [-1] * len(tokens)  # Initialize labels with -1 (no change)

    # Determine which tokens can be masked
    candidate_indices = [
        i for i, token in enumerate(tokens) 
        if token not in [tokenizer.cls_token, tokenizer.sep_token]
    ]
    rng.shuffle(candidate_indices)
    num_masked = min(max_predictions_per_seq, len(candidate_indices) * 15 // 100)
    
    for index in candidate_indices[:num_masked]:
        random_choice = rng.random()
        # 80% replace with [MASK], 10% random token, 10% unchanged
        if random_choice < 0.8:
            output_tokens[index] = tokenizer.mask_token
        elif random_choice < 0.9:
            output_tokens[index] = random.choice(list(tokenizer.vocab.keys()))
        
        output_labels[index] = tokenizer.convert_tokens_to_ids(tokens[index])

    return output_tokens, output_labels

def truncate_and_process(tokens_a, tokens_b, max_seq_length, tokenizer, max_predictions_per_seq, instances, rng, is_random_next):
    # Truncate tokens_a and tokens_b if their combined length is too long
    while len(tokens_a) + len(tokens_b) + 3 > max_seq_length:
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()

    tokens = ['[CLS]'] + tokens_a + ['[SEP]'] + tokens_b + ['[SEP]']
    segment_ids = [0] * (len(tokens_a) + 2) + [1] * (len(tokens_b) + 1)

    masked_tokens, masked_labels = mask_tokens(tokens, tokenizer, max_predictions_per_seq, rng)

    # Convert masked_tokens to IDs
    token_ids = tokenizer.convert_tokens_to_ids(masked_tokens)  # Ensure this returns integers
    instance = TrainingInstance(
        tokens=token_ids,
        segment_ids=segment_ids,
        masked_lm_positions=[i for i, label in enumerate(masked_labels) if label != -1],
        masked_lm_labels=[label for label in masked_labels if label != -1],
        is_random_next=int(is_random_next)
    )
    # instance = {
    #     'tokens': token_ids,
    #     'segment_ids': segment_ids,
    #     'masked_lm_positions': [i for i, label in enumerate(masked_labels) if label != -1],
    #     'masked_lm_labels': [label for label in masked_labels if label != -1],
    #     'is_random_next': int(is_random_next)
    # }
    instances.append(instance)



In [84]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vocab_size = tokenizer.vocab_size  # Smaller vocabulary size for simplicity
print(vocab_size)
num_layers = 4  # Fewer layers
d_model = 768  # Smaller dimensionality
num_heads = 4
dff = 3072
segment_size = 2
max_seq_length = 32
max_predictions_per_seq=5
batch_size = 16
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

30522


In [85]:
def create_bert_pretraining_instances_in_chunks(file_path, chunk_size=1048576, 
                          doc_boundary_pattern=r'ARTICLE-\d+-\d+-https:\/\/\S+',
                         test_print=10, max_seq_length=max_seq_length, 
                         max_predictions_per_seq=max_predictions_per_seq, 
                         dupe_factor=5, random_seed=12345, nsp_enabled=True,
                                               tokenizer=tokenizer):
    # tokenizer = BertTokenizer.from_pretrained(args.vocab_file, do_lower_case=args.do_lower_case)  ## is for custom vocab
    # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') ### This loads the default vocab for bert pre-training
    rng = random.Random(random_seed)
    buffer = ''
    instances = []    
    with open(file_path, 'r', encoding='utf-8') as file:
        while True:
            chunk = file.read(chunk_size)
            if not chunk:
                break
            buffer += chunk
            documents = re.split(doc_boundary_pattern, buffer, flags=re.MULTILINE)            
            if documents and not re.match(doc_boundary_pattern, documents[-1]):
                buffer = documents.pop()
            else:
                buffer = ''            
            for doc in documents:
                if not doc.strip():
                    continue
                tokenized_doc = tokenizer.tokenize(doc)
                sequences = [tokenized_doc[i:i+max_seq_length] for i in range(0, len(tokenized_doc), max_seq_length)]                
                for j in range(len(sequences) - 1):
                    tokens_a = sequences[j]
                    if rng.random() > 0.5 or not nsp_enabled:
                        is_random_next = True
                        tokens_b = sequences[rng.randint(0, len(sequences) - 1)]
                    else:
                        is_random_next = False
                        tokens_b = sequences[j + 1]

                    truncate_and_process(tokens_a, tokens_b, max_seq_length, tokenizer, max_predictions_per_seq, instances, rng, is_random_next)

                    if test_print > 0:
                        print(f"Tokens A: {tokens_a}, len: {len(tokens_a)}")
                        print(f"Tokens B: {tokens_b[:10]}")
                        print(f"Is random next: {is_random_next}\n")
                        test_print -= 1
                        if test_print == 0:
                            return instances
    return instances  # Return all instances for further processing or training
file_path = 'wiki_articles_with_seperator.txt'
res = create_bert_pretraining_instances_in_chunks(file_path)

Tokens A: ['ana', '##rch', '##ism', 'is', 'a', 'political', 'philosophy', 'and', 'movement', 'that', 'is', 'sc', '##ept', '##ical', 'of'], len: 15
Tokens B: ['for', 'the', 'abolition', 'of', 'the', 'state', ',', 'which', 'it', 'holds']
Is random next: False

Tokens A: ['for', 'the', 'abolition', 'of', 'the', 'state', ',', 'which', 'it', 'holds', 'to', 'be', 'unnecessary', ','], len: 14
Tokens B: ['the', 'far', '##thest', 'left', 'of', 'the', 'political', 'spectrum', ',', 'it']
Is random next: False

Tokens A: ['the', 'far', '##thest', 'left', 'of', 'the', 'political', 'spectrum', ',', 'it', 'is', 'usually', 'described', 'alongside', 'communal'], len: 15
Tokens B: ['and', 'has', 'a', 'strong', 'historical', 'association', 'with', 'anti', '-', 'capitalism']
Is random next: False

Tokens A: ['and', 'has', 'a', 'strong', 'historical', 'association', 'with', 'anti', '-', 'capitalism', 'and', 'socialism', '.', 'humans'], len: 14
Tokens B: ['fiction', 'due', 'to', 'the', 'fact', 'that', 'the'

In [86]:
# print(res[0])
# # d = res[0]
# print(d.__dict__)

In [87]:
def save_instances_as_parquet(instances, file_path, num_instances=1000, small=False):
    if small:
        instances = instances[:num_instances]
    data = {
        'input_ids': [],
        'segment_ids': [],
        'masked_lm_positions': [],
        'mlm_labels': [],
        'nsp_labels': []
    }
    for instance in instances:
        data['input_ids'].append(instance.tokens)
        data['segment_ids'].append(instance.segment_ids)
        data['masked_lm_positions'].append(instance.masked_lm_positions)
        data['mlm_labels'].append(instance.masked_lm_labels)
        data['nsp_labels'].append(instance.is_random_next)
    df = pd.DataFrame(data)
    df.to_parquet(file_path, engine='pyarrow')
instances = create_bert_pretraining_instances_in_chunks(file_path, test_print=0)
save_instances_as_parquet(instances, 'pretraining_bert_data.parquet')

In [88]:
def load_dataset(file_path, batch_size=32):
    df = pd.read_parquet(file_path)
    tensor_dict = {}
    for col in df.columns:
        if isinstance(df[col].values[0], list) or isinstance(df[col].values[0], np.ndarray):
            # Handle list or array: pad sequences and convert to tensor
            padded_array = tf.keras.preprocessing.sequence.pad_sequences(df[col].tolist(), padding='post', dtype='int32')
            tensor_dict[col] = tf.convert_to_tensor(padded_array, dtype=tf.int32)
        else:
            # Convert scalar values directly to tensor
            tensor_dict[col] = tf.convert_to_tensor(df[col].to_numpy(dtype=np.int32), dtype=tf.int32)

    # Split the tensors into inputs and labels
    inputs = {k: tensor_dict[k] for k in ['input_ids', 'segment_ids']}
    labels = {k: tensor_dict[k] for k in ['masked_lm_positions', 'mlm_labels', 'nsp_labels']}
    # Combine into a single dataset
    dataset = tf.data.Dataset.from_tensor_slices((inputs, labels))
    dataset = dataset.batch(batch_size)
    return dataset


train_dataset = load_dataset('pretraining_bert_data.parquet', batch_size=batch_size)

for inputs, labels in train_dataset.take(1):
    print("Inputs:", inputs)
    print("Labels:", labels)
    break


Inputs: {'input_ids': <tf.Tensor: shape=(16, 32), dtype=int32, numpy=
array([[  101,  9617,   103,  2964,  2003,  1037,  2576,  4695,  1998,
         2929,  2008,  2003,  8040,   103,  7476,   103,   102,  2005,
          103, 15766,  1997,  1996,  2110,  1010,  2029,  2009,  4324,
         2000,  2022, 14203,  1010,   102],
       [  101,  2005,  1996, 15766,  1997,   103,  2110,  1010,   103,
         2009,  4324,  2000,   103,   103,  1010,   102,  1996,  2521,
        20515,  2187,  1997,  1996,  2576,  8674,  1010,  2009,  2003,
         2788,  2649,  4077, 15029,   102],
       [  101,  1996,  2521, 20515,   103,  1997,  1996,  2576,  8674,
         1010,  2009,  2003,  2788,  2649,  4077, 15029,   102,  1998,
          103,  1037,  2844,  3439,   103,  2007,  3424,   103, 16498,
         1998, 14649,  1012,  4286,   102],
       [  101,  1998,  2038,  1037,  2844,  3439,  2523,  2007,  3424,
         1011, 16498,  1998, 14649,  1012,  4286,   102,  4349,  2349,
          103,  1

In [89]:
### Convert these articles to pretraining data 
# !python create_pretraining_data.py --vocab_file vocab.txt --input_text input_text.txt --output_tfrecord output.tfrecord --do_lower_case --nsp


In [90]:
single_test_instance = iter(train_dataset.take(1)).next()
single_input_tuple = single_test_instance[0]['input_ids'], single_test_instance[0]['segment_ids']
# print(single_test_instance)
print(single_test_instance[0]['input_ids'][0])
print(single_test_instance[1]['masked_lm_positions'][0])
print(single_test_instance[1]['mlm_labels'][0])

tf.Tensor(
[  101  9617   103  2964  2003  1037  2576  4695  1998  2929  2008  2003
  8040   103  7476   103   102  2005   103 15766  1997  1996  2110  1010
  2029  2009  4324  2000  2022 14203  1010   102], shape=(32,), dtype=int32)
tf.Tensor([ 2 13 15 18], shape=(4,), dtype=int32)
tf.Tensor([11140 23606  1997  1996], shape=(4,), dtype=int32)


In [91]:
class PositionalAndSegmentEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, segment_size, d_model, max_pos=2048, pos_dropout=0.1, **kwargs):
        super().__init__(**kwargs)  # Initialize the superclass (Layer)
        self.d_model = d_model  # Store the dimensionality of the model embeddings
        self.token_embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True)
        self.segment_embedding = tf.keras.layers.Embedding(segment_size, d_model)
        self.pos_encoding = encode_pos_sin_cosine(max_pos, d_model, debug=False)
        self.dropout = tf.keras.layers.Dropout(pos_dropout)

    def compute_mask(self, inputs, *args, **kwargs):
        # Assuming the input structure is a tuple of (tokens, segments)
        token_inputs, _ = inputs
        return self.token_embedding.compute_mask(token_inputs, *args, **kwargs)

    def call(self, inputs, training=False):
        # Expect inputs to be a tuple (token_inputs, segment_inputs)
        token_inputs, segment_inputs = inputs
        tokens = self.token_embedding(token_inputs)  # Token embeddings
        segments = self.segment_embedding(segment_inputs)  # Segment embeddings       
        x = tokens + segments
        # Scale the embeddings by the square root of the embedding dimension size
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        # Add positional encoding to the combined embeddings, sliced to match the input length
        length = tf.shape(x)[1]
        # pos_encodings = tf.reshape(self.pos_encoding, (1, -1, self.d_model))[:, :length, :]
        pos_encodings = tf.cast(tf.reshape(self.pos_encoding, (1, -1, self.d_model))[:, :tf.shape(x)[1], :], tf.float32)
        x += pos_encodings
        x = self.dropout(x, training=training)
        return x

embedding_layer = PositionalAndSegmentEmbedding(vocab_size=vocab_size, segment_size=2, d_model=256)

# Extract a single batch from the dataset
for inputs, labels in train_dataset.take(1):
    # The inputs dictionary contains 'input_ids' and 'segment_ids'
    input_ids = inputs['input_ids']
    segment_ids = inputs['segment_ids']

    # Call the embedding layer
    embeddings = embedding_layer((input_ids, segment_ids))

    # Print the output shape
    print("Output shape:", embeddings.shape)
    print("Output shape:", embeddings._keras_mask)
    break

Output shape: (16, 32, 256)
Output shape: tf.Tensor(
[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True
   

In [92]:
class TransformerEncoderV4(TransformerEncoderV3):
    def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, segment_size, max_pos=2048, pos_dropout=0.1, **kwargs):
        super(TransformerEncoderV4, self).__init__(num_layers=num_layers, d_model=d_model, num_heads=num_heads, dff=dff, vocab_size=vocab_size, max_pos=max_pos, **kwargs)
        # Use the custom embedding layer that handles tokens, segments, and positional encodings
        self.embedding_layer = PositionalAndSegmentEmbedding(vocab_size, segment_size, d_model, max_pos, pos_dropout)

    def call(self, inputs, training=False):
        input_ids, segment_ids = inputs
        # The embedding layer now handles everything including token, segment, and positional embeddings
        x = self.embedding_layer((input_ids, segment_ids), training=training)
        x = self.enc_layers_0(x, training=training)
        for i in range(self.remaining_layers):
            x = self.enc_layers[i](x, training=training)
        return x
tren = TransformerEncoderV4(num_layers, d_model, num_heads, dff, vocab_size, segment_size, max_pos=max_seq_length)

encoder_out = tren(single_input_tuple)
print(encoder_out.shape)

(16, 32, 768)


In [117]:
# class BERT(tf.keras.Model):
#     def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, segment_size, max_seq_length=128, rate=0.1):
#         super(BERT, self).__init__()
#         self.encoder = TransformerEncoderV4(num_layers=num_layers, d_model=d_model, num_heads=num_heads,
#                                             dff=dff, vocab_size=vocab_size, segment_size=segment_size,
#                                             max_pos=max_seq_length, pos_dropout=rate)
#         self.mlm_dense = tf.keras.layers.Dense(vocab_size, activation='softmax')  # Ensures output shape is [batch, seq_length, vocab_size]
#         self.nsp_dense = tf.keras.layers.Dense(1, activation='sigmoid')   
    
#     def call(self, inputs, training=False):
#         x = self.encoder((inputs['input_ids'], inputs['segment_ids']), training=training)
#         mlm_output = self.mlm_dense(x)  # Check shapes here
#         nsp_output = self.nsp_dense(x[:, 0, :])
#         return {'mlm_output': mlm_output, 'nsp_output': nsp_output}

# bert_model = BERT(num_layers, d_model, num_heads, 
#                   dff, vocab_size, segment_size)
# bert_out = bert_model(single_test_instance[0])
# print(bert_out)

{'mlm_output': <tf.Tensor: shape=(16, 32, 30522), dtype=float32, numpy=
array([[[2.9132796e-05, 4.1347856e-05, 4.0508101e-05, ...,
         3.1462605e-05, 2.2753575e-05, 2.6322496e-05],
        [2.2634018e-05, 4.1381449e-05, 3.8698367e-05, ...,
         3.4007335e-05, 2.3044084e-05, 3.1454834e-05],
        [2.6919319e-05, 4.7942154e-05, 4.1582123e-05, ...,
         3.2779881e-05, 2.1349722e-05, 3.0214544e-05],
        ...,
        [3.4179207e-05, 4.2355263e-05, 3.5524554e-05, ...,
         2.5386791e-05, 2.6338388e-05, 2.3977718e-05],
        [4.3079119e-05, 3.3040353e-05, 3.4941444e-05, ...,
         2.5725280e-05, 2.5267344e-05, 2.2713668e-05],
        [3.5298261e-05, 4.1672167e-05, 3.6357946e-05, ...,
         2.6349733e-05, 2.5629966e-05, 2.6083797e-05]],

       [[2.7661485e-05, 4.0958759e-05, 4.0110586e-05, ...,
         3.1388656e-05, 2.4603614e-05, 2.8394028e-05],
        [2.2942872e-05, 4.1769668e-05, 3.7569400e-05, ...,
         3.5041659e-05, 2.2891305e-05, 2.5415140e-05],
 

In [144]:
class BERT(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, segment_size, max_seq_length=128, rate=0.1):
        super(BERT, self).__init__()
        self.encoder = TransformerEncoderV4(num_layers=num_layers, d_model=d_model, num_heads=num_heads,
                                            dff=dff, vocab_size=vocab_size, segment_size=segment_size,
                                            max_pos=max_seq_length, pos_dropout=rate)
        self.mlm_dense_transform = tf.keras.layers.Dense(d_model, activation='gelu')  # Transform layer for MLM
        self.mlm_layer_norm = tf.keras.layers.LayerNormalization(epsilon=1e-12)  # Layer normalization for MLM
        self.mlm_dense = tf.keras.layers.Dense(vocab_size)  # Ensures output shape is [batch, seq_length, vocab_size]
        self.nsp_dense = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs, training=False):
        x = self.encoder((inputs['input_ids'], inputs['segment_ids']), training=training)
        
        # Apply dense transformation and layer normalization for MLM
        mlm_output = self.mlm_dense_transform(x)
        mlm_output = self.mlm_layer_norm(mlm_output)
        mlm_output = self.mlm_dense(mlm_output)
        
        nsp_output = self.nsp_dense(x[:, 0, :])
        return {'mlm_output': mlm_output, 'nsp_output': nsp_output}

# Instantiate and compile the BERT model
bert_model = BERT(num_layers, d_model, num_heads, 
                  dff, vocab_size, segment_size)
bert_out = bert_model(single_test_instance[0])
print(bert_out['mlm_output'].shape  )
print(bert_out['nsp_output'].shape  )

(16, 32, 30522)
(16, 1)


In [146]:
learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)
# optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)
loss_object_nsp = tf.keras.losses.BinaryCrossentropy(from_logits=True)


def compute_mlm_loss(masked_positions, masked_labels, logits):
    # Gather the logits at the masked positions
    masked_logits = tf.gather(logits, masked_positions, batch_dims=1, axis=1) 
    # tf.print("masked_positions:", masked_positions[0], summarize=-1)
    # tf.print("masked_labels:", masked_labels[0], summarize=-1)
    # tf.print("masked_logits:", masked_logits[0].shape, summarize=-1)
    # Ensure that the masked_labels used here are the correct length and match the number of masked_positions
    mlm_loss = tf.keras.losses.sparse_categorical_crossentropy(masked_labels, masked_logits, from_logits=True)
    # tf.print("mlm_loss:", mlm_loss[0], summarize=-1)
    # Reduce mean across batches if needed or sum as appropriate
    return tf.reduce_mean(mlm_loss)
    
def compute_nsp_loss(labels, logits):
    return tf.keras.losses.binary_crossentropy(labels, logits, from_logits=True)

In [147]:
@tf.function
def train_step(inputs, labels, task='both'):
    with tf.GradientTape() as tape:        
        predictions = bert_model(inputs, training=True)  # Predictions will have 'mlm_output' and 'nsp_output'
        # Compute the MLM loss using the positions and labels
        loss_mlm = compute_mlm_loss(labels['masked_lm_positions'], 
                                    labels['mlm_labels'], predictions['mlm_output'])
        # NSP loss remains the same
        loss_nsp = loss_object_nsp(labels['nsp_labels'], predictions['nsp_output'])
        if task == 'nsp':
            total_loss = loss_nsp
        elif task == 'mlm':
            total_loss = loss_mlm
        else:
            total_loss = loss_mlm + loss_nsp
    gradients = tape.gradient(total_loss, bert_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, bert_model.trainable_variables))
    # tf.print("Total Loss:", total_loss, "MLM Loss:", loss_mlm, "NSP Loss:", loss_nsp)
    return total_loss, loss_mlm, loss_nsp


train_dataset = train_dataset.repeat()
epochs = 100
for epoch in range(epochs):
    step = 0
    for inputs, labels in train_dataset.take(1):
        # tf.print("input_ids:", inputs['input_ids'][0], summarize=-1)
        loss_values = train_step(inputs, labels, task='mlm')
        if step % 10 == 0:
            print(f"Epoch {epoch + 1}, Step {step}, Total Loss: {loss_values[0].numpy():.4f}, MLM Loss: {loss_values[1].numpy():.4f}, NSP Loss: {loss_values[2].numpy():.4f}")
        step += 1
# for epoch in range(epochs):
#     step = 0
#     # Create a new iterator for each epoch
#     dataset_iter = iter(train_dataset.take(10))
#     while True:
#         try:
#             inputs, labels = next(dataset_iter)
#             loss_values = train_step(inputs, labels, task='mlm')
#             if step % 10 == 0:
#                 print(f"Epoch {epoch + 1}, Step {step}, Total Loss: {loss_values[0].numpy():.4f}, MLM Loss: {loss_values[1].numpy():.4f}, NSP Loss: {loss_values[2].numpy():.4f}")
#             step += 1
#             # Break after one batch to simulate take(1)
#             # break
#         except StopIteration:
#             break

Epoch 1, Step 0, Total Loss: 10.3523, MLM Loss: 10.3523, NSP Loss: 0.7588


2024-05-16 11:05:27.820836: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2, Step 0, Total Loss: 10.3740, MLM Loss: 10.3740, NSP Loss: 0.8065


2024-05-16 11:05:29.159169: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 3, Step 0, Total Loss: 10.3871, MLM Loss: 10.3871, NSP Loss: 0.8478


...

2024-05-16 11:07:42.547898: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 99, Step 0, Total Loss: 8.5649, MLM Loss: 8.5649, NSP Loss: 0.7803


2024-05-16 11:07:44.111065: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 100, Step 0, Total Loss: 8.5479, MLM Loss: 8.5479, NSP Loss: 0.7651


2024-05-16 11:07:45.505561: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [148]:
epochs = 100
for epoch in range(epochs):
    step = 0
    for inputs, labels in train_dataset.take(1):
        # tf.print("input_ids:", inputs['input_ids'][0], summarize=-1)
        loss_values = train_step(inputs, labels, task='both')
        if step % 10 == 0:
            print(f"Epoch {epoch + 1}, Step {step}, Total Loss: {loss_values[0].numpy():.4f}, MLM Loss: {loss_values[1].numpy():.4f}, NSP Loss: {loss_values[2].numpy():.4f}")
        step += 1

Epoch 1, Step 0, Total Loss: 9.2853, MLM Loss: 8.5035, NSP Loss: 0.7818


2024-05-16 16:11:12.980846: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 2, Step 0, Total Loss: 9.2618, MLM Loss: 8.4925, NSP Loss: 0.7693


2024-05-16 16:11:14.137424: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 3, Step 0, Total Loss: 9.2957, MLM Loss: 8.4526, NSP Loss: 0.8431


...

2024-05-16 16:13:08.996257: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 99, Step 0, Total Loss: 5.3376, MLM Loss: 5.3368, NSP Loss: 0.0009


2024-05-16 16:13:10.164782: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 100, Step 0, Total Loss: 5.2939, MLM Loss: 5.2930, NSP Loss: 0.0009


2024-05-16 16:13:11.312709: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [153]:
fit_model = bert_model
COLAB = False
if not COLAB:
      model_path= '/mnt/d/MyDev/attention/transformerlab/bert/models'
model_full_name = os.path.join(model_path, 'bert_pretrained_1.weights.h5')
fit_model.save_weights(model_full_name)
with open(model_path + '/bert_pt_1_config.pkl', 'wb') as config_file:
        pickle.dump(fit_model.get_config(), config_file)
with open(model_path + '/bert_pt_1_config.pkl', 'rb') as config_file:
        fit_model_config = pickle.load(config_file)
loaded_fit_model = BERT(num_layers, d_model, num_heads, 
                  dff, vocab_size, segment_size)

loaded_fit_model_out = loaded_fit_model(single_test_instance[0])
loaded_fit_model.load_weights(model_full_name)
print(fit_model.summary())
print(loaded_fit_model.summary())

Model: "bert_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ transformer_encoder_v4_14       │ ?                      │    52,238,592 │
│ (TransformerEncoderV4)          │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_457 (Dense)               │ ?                      │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_189         │ ?                      │         1,536 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_458 (Dense)               │ ?                      │    23,471,418 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_459 (Dense)               │ ?                      │           769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 76,302,907 (291.07 MB)

 Trainable params: 76,302,907 (291.07 MB)

 Non-trainable params: 0 (0.00 B)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ transformer_encoder_v4_16       │ ?                      │    52,238,592 │
│ (TransformerEncoderV4)          │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_535 (Dense)               │ ?                      │       590,592 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_223         │ ?                      │         1,536 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_536 (Dense)               │ ?                      │    23,471,418 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_537 (Dense)               │ ?                      │           769 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 76,302,907 (291.07 MB)

 Trainable params: 76,302,907 (291.07 MB)

 Non-trainable params: 0 (0.00 B)

None


In [96]:
# def parse_tfrecord(serialized_example):
#     feature_description = {
#         'input_ids': tf.io.VarLenFeature(tf.int64),
#         'segment_ids': tf.io.VarLenFeature(tf.int64),
#         'masked_lm_positions': tf.io.VarLenFeature(tf.int64),
#         'masked_lm_labels': tf.io.VarLenFeature(tf.int64),
#         'next_sentence_labels': tf.io.FixedLenFeature([], tf.int64)
#     }
#     # print(serialized_example)
#     example = tf.io.parse_single_example(serialized_example, feature_description)
#     # print(example)
#     input_ids = tf.cast(example['input_ids'], tf.int32)
#     input_ids = tf.sparse.to_dense(input_ids)
#     segment_ids = tf.cast(example['segment_ids'], tf.int32)
#     segment_ids = tf.sparse.to_dense(segment_ids)
#     masked_lm_positions = tf.sparse.to_dense(example['masked_lm_positions'])
#     masked_lm_labels = tf.sparse.to_dense(example['masked_lm_labels'])
#     next_sentence_labels = tf.cast(example['next_sentence_labels'], tf.int32)
#     inputs = {'input_ids': input_ids, 'segment_ids': segment_ids}
#     labels = {'masked_lm_positions': masked_lm_positions,
#               'mlm_labels': masked_lm_labels, 'nsp_labels': next_sentence_labels}
#     return (inputs, labels)

# def load_dataset(filepath, batch_size):
#     raw_dataset = tf.data.TFRecordDataset(filepath)
#     parsed_dataset = raw_dataset.map(parse_tfrecord)
#     # Define padding shapes for each component of the dataset
#     padded_shapes = ({
#         'input_ids': [None],  # Dynamic padding for input_ids
#         'segment_ids': [None]  # Dynamic padding for segment_ids
#     }, {
#         'masked_lm_positions': [None],  # Dynamic padding for positions
#         'mlm_labels': [None],  # Dynamic padding for mlm labels
#         'nsp_labels': []  # No padding needed for scalar labels
#     })

#     # Use padded_batch to handle variable sequence lengths
#     batched_dataset = parsed_dataset.padded_batch(batch_size, padded_shapes=padded_shapes)
#     # batched_dataset = parsed_dataset.batch(batch_size)    
#     return batched_dataset

# # Usage
# batch_size = 32
# train_dataset = load_dataset('output.tfrecord', batch_size)

In [97]:
# def load_and_print_dataset(filepath):
#     raw_dataset = tf.data.TFRecordDataset(filepath)
#     for i, raw_record in enumerate(raw_dataset):  # Adjust the number based on how many you want to check
#         # print("Raw record:", raw_record.numpy())
#         try:
#             example = tf.io.parse_single_example(
#                 raw_record,
#                 {
#                     # 'input_ids': tf.io.FixedLenFeature([128], tf.int64),
#                     'input_ids': tf.io.VarLenFeature(tf.int64),
#                     'segment_ids': tf.io.VarLenFeature(tf.int64),
#                     'masked_lm_positions': tf.io.VarLenFeature(tf.int64),
#                     'masked_lm_labels': tf.io.VarLenFeature(tf.int64),
#                     'next_sentence_labels': tf.io.FixedLenFeature([], tf.int64)
#                 }
#             )
#             # print("Parsed example:", example)
#         except Exception as e:
#             print(f"Failed to parse record {i}: {e}")
#             print("Raw record:", raw_record.numpy())
#             break  # Or continue based on how you want to handle errors

# # Example usage
# load_and_print_dataset('output.tfrecord')


In [98]:
# def create_bert_pretraining_instances_in_chunks(file_path, chunk_size=1048576, 
#                           doc_boundary_pattern=r'ARTICLE-\d+-\d+-https:\/\/\S+',
#                          test_print=2):   
#     '''
#     the chunk can read more than one documents and hence the buffer can hold more that one doc . Hence the documents can also hold more than one doc
#     but when chunk is smaller than the smallest  doc in file, the documents is essentially end up hoding one doc at a time 
#     '''
#     tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#     buffer = ''
#     with open(file_path, 'r', encoding='utf-8') as file:
#         while True:
#             chunk = file.read(chunk_size)
#             if not chunk:
#                 break
#             buffer += chunk
#             # Split buffer by document boundary and keep the content only
#             documents = re.split(doc_boundary_pattern, buffer, flags=re.MULTILINE)            
#             # If the last part might not be a complete document, keep it in the buffer
#             if documents and not re.match(doc_boundary_pattern, documents[-1]):
#                 buffer = documents.pop()
#             else:
#                 buffer = ''            
#             # Process each document found in this chunk
#             for i, doc in enumerate(documents):
#                 if not doc.strip():  # Skip any empty results from split
#                     continue
#                 sentences = nltk.sent_tokenize(doc)
#                 tokens = tokenizer.tokenize(doc)
#                 if test_print > 0:
#                     print(f"First few tokens: {tokens[:10]}") 
#                     print(f"First sentence: {sentences[:3] if sentences else 'No content'}\n")
#                 if i == test_print: return
# file_path = 'wiki_articles_with_seperator.txt'
# create_bert_pretraining_instances_in_chunks(file_path)

In [99]:
# def create_bert_pretraining_instances_in_chunks(file_path, chunk_size=1048576, 
#                           doc_boundary_pattern=r'ARTICLE-\d+-\d+-https:\/\/\S+',
#                          test_print=10, max_seq_length=128, max_predictions_per_seq=20, 
#                          dupe_factor=5, random_seed=12345, nsp_enabled=True):
#     tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#     buffer = ''
#     rng = random.Random(random_seed)
#     instances = []
#     with open(file_path, 'r', encoding='utf-8') as file:
#         while True:
#             chunk = file.read(chunk_size)
#             if not chunk:
#                 break
#             buffer += chunk
#             documents = re.split(doc_boundary_pattern, buffer, flags=re.MULTILINE)
#             if documents and not re.match(doc_boundary_pattern, documents[-1]):
#                 buffer = documents.pop()
#             else:
#                 buffer = ''
            
#             for i, doc in enumerate(documents):
#                 if not doc.strip():
#                     continue
#                 tokenized_doc = tokenizer.tokenize(doc)
#                 # Split tokenized document into sequences, not implemented here, assumes tokenized_doc is already split
#                 sequences = [tokenized_doc[i:i+max_seq_length] for i in range(0, len(tokenized_doc), max_seq_length)]
#                 for j in range(len(sequences) - 1):
#                     tokens_a = sequences[j]
#                     if rng.random() > 0.5 or not nsp_enabled:
#                         is_random_next = True
#                         # Randomly pick any sequence from any document
#                         tokens_b = sequences[rng.randint(0, len(sequences) - 1)]
#                     else:
#                         is_random_next = False
#                         # Ensure tokens_b is the next sequence in the same document
#                         tokens_b = sequences[j + 1]
                    
#                     if test_print > 0:
#                         print(f"Tokens A: {tokens_a}, len:: {len(tokens_a) }")
#                         print(f"Tokens B: {tokens_b[:10]}")
#                         print(f"Is random next: {is_random_next}\n")
#                         test_print -= 1
#                         if test_print == 0:
#                             return  # Early exit for test purposes

# file_path = 'wiki_articles_with_seperator.txt'
# res = create_bert_pretraining_instances_in_chunks(file_path)

In [100]:
# class Human:
#     def __init__(self, name, legs=2):
#         self.name = name
#         self.legs = legs

# class MaleHuman(Human):
#     def __init__(self, *args, **kwargs):
#         super(MaleHuman, self).__init__(*args, **kwargs)
#         # self.beard = beard

# m1 = MaleHuman('Bhujay')
# m1.name

In [101]:
# dataset = dataset.padded_batch(
    #     batch_size, 
    #     padded_shapes={'input_ids': [None], 
    #                    'segment_ids': [128], 
    #                    'masked_lm_positions': [None], 
    #                    'masked_lm_labels': [None], 
    #                    'next_sentence_labels': []})

In [102]:

# try:
#     for idx, (inputs, labels) in enumerate(train_dataset):
#         print(f"instance {idx + 1}:")
#         # print(f"Inputs: {inputs}")
#         # print(f"Labels: {labels}")
#         if idx == 10:  # Limit to 2 batches for demonstration
#             break
# except Exception as e:
#     print(f"Error while processing the dataset: {e}")

In [103]:
# def test_dataset(filepath):
#     # dataset = load_dataset(filepath, batch_size=16)  # Using batch size of 1 for simplicity
#     try:
#         for i, (inputs, labels) in enumerate(train_dataset):        
#             print(f"Successfully parsed entry {i+1}")        
#             if i == 100:  # Optionally limit the number of entries to check
#                 break
#     except Exception as e:
#             print(f'record: {i}  , Err: {e}')

# test_dataset('output.tfrecord')

In [104]:
# def parse_tfrecord(serialized_example):
#     feature_description = {
#         'input_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
#         'segment_ids': tf.io.FixedLenFeature([max_seq_length], tf.int64),
#         'masked_lm_positions': tf.io.VarLenFeature(tf.int64),
#         'masked_lm_labels': tf.io.VarLenFeature(tf.int64),
#         'next_sentence_labels': tf.io.FixedLenFeature([], tf.int64)
#     }
#     try:
#         example = tf.io.parse_single_example(serialized_example, feature_description)    
#         input_ids = tf.cast(example['input_ids'], tf.int32)
#         segment_ids = tf.cast(example['segment_ids'], tf.int32)
#         # segment_ids = tf.sparse.to_dense(segment_ids, default_value=0)  # Default to 0
#         # segment_ids = tf.reshape(segment_ids, [max_seq_length]) 
#         masked_lm_positions = tf.sparse.to_dense(example['masked_lm_positions'])
#         masked_lm_labels = tf.sparse.to_dense(example['masked_lm_labels'])   
#         next_sentence_labels = tf.cast(example['next_sentence_labels'], tf.int32)
#         inputs = {'input_ids': input_ids, 'segment_ids': segment_ids, }
#         labels = {'masked_lm_positions': masked_lm_positions, 
#                   'mlm_labels': masked_lm_labels, 'nsp_labels': next_sentence_labels}
#     except Exception as e:
#         print(f"Failed to parse example: {e}")
#         # Return dummy/default data to allow pipeline to continue
#         inputs = {
#             'input_ids': tf.constant([0]*args.max_seq_length, dtype=tf.int32),
#             'segment_ids': tf.constant([0]*args.max_seq_length, dtype=tf.int32)
#             }
#         labels = {
#             'masked_lm_positions': tf.constant([-1]*20, dtype=tf.int32),  # Adjust 20 to your max_predictions_per_seq
#             'masked_lm_labels': tf.constant([-1]*20, dtype=tf.int32),
#             'nsp_labels': tf.constant([0], dtype=tf.int32)
#             } 
#         return (None, None)
#     return inputs, labels




# def load_dataset(filepath, batch_size):
#     raw_dataset = tf.data.TFRecordDataset(filepath)
#     parsed_dataset = raw_dataset.map(parse_tfrecord)
#     filtered_dataset = parsed_dataset.filter(lambda x: x[0] is not None and x[1] is not None)
#     batched_dataset = filtered_dataset.batch(batch_size)
#     return batched_dataset
# # Usage
# batch_size = 16
# train_dataset = load_dataset('output.tfrecord', batch_size)

# single_test_instance = iter(train_dataset.take(1)).next()
# single_input_tuple = single_test_instance[0]['input_ids'], single_test_instance[0]['segment_ids']
# print(single_test_instance)
# print()
# # print(single_input_tuple)

In [105]:
# def _parse_function(proto):
#     # Define your tfrecord again. It must be the same as the one used for saving your data.
#     feature_description = {
#         'input_ids': tf.io.FixedLenFeature([128], tf.int64),  # Assuming input_ids are of length 128
#         'segment_ids': tf.io.FixedLenFeature([128], tf.int64),  # Assuming segment_ids are of length 128
#         'masked_lm_positions': tf.io.VarLenFeature(tf.int64),
#         'masked_lm_labels': tf.io.VarLenFeature(tf.int64),
#         'next_sentence_labels': tf.io.FixedLenFeature([], tf.int64),
#     }

#     # Load one example
#     parsed_features = tf.io.parse_single_example(proto, feature_description)
    
#     # Turn your sparse array into a dense array with default values as 0
#     parsed_features['masked_lm_positions'] = tf.sparse.to_dense(parsed_features['masked_lm_positions'], default_value=0)
#     parsed_features['masked_lm_labels'] = tf.sparse.to_dense(parsed_features['masked_lm_labels'], default_value=0)

#     return parsed_features
# # Read the TFRecord file
# def load_dataset(file_path):
#     dataset = tf.data.TFRecordDataset(file_path)
#     dataset = dataset.map(_parse_function)  # Parse the record into tensors.
#     return dataset

# # Path to the TFRecord file
# tfrecord_file_path = 'output.tfrecord'

# # Load the dataset
# parsed_dataset = load_dataset(tfrecord_file_path)
# # Display a few examples from the dataset
# for parsed_record in parsed_dataset.take(2):  # Only take first 5 examples
#     print('Input IDs:', parsed_record['input_ids'].numpy())
#     print('Segment IDs:', parsed_record['segment_ids'].numpy())
#     print('Masked LM Positions:', parsed_record['masked_lm_positions'].numpy())
#     print('Masked LM Labels:', parsed_record['masked_lm_labels'].numpy())
#     print('Next Sentence Label:', parsed_record['next_sentence_labels'].numpy())
#     print('---')


In [106]:
# def parse_tfrecord(serialized_example):
#     feature_description = {
#         'input_ids': tf.io.FixedLenFeature([128], tf.int64),
#         'segment_ids': tf.io.FixedLenFeature([128], tf.int64),
#         'masked_lm_positions': tf.io.VarLenFeature(tf.int64),
#         'masked_lm_labels': tf.io.VarLenFeature(tf.int64),
#         'next_sentence_labels': tf.io.FixedLenFeature([], tf.int64)
#     }
#     example = tf.io.parse_single_example(serialized_example, feature_description)

#     input_ids = tf.cast(example['input_ids'], tf.int32)
#     segment_ids = tf.cast(example['segment_ids'], tf.int32)
#     masked_lm_positions = tf.sparse.to_dense(example['masked_lm_positions'])
#     masked_lm_labels = tf.sparse.to_dense(example['masked_lm_labels'])
#     next_sentence_labels = tf.cast(example['next_sentence_labels'], tf.int32)

#     inputs = {'input_ids': input_ids, 'segment_ids': segment_ids}
#     labels = {'mlm_output': masked_lm_labels, 'nsp_output': next_sentence_labels}

#     return inputs, labels

In [107]:
# optimizer = Adam(learning_rate=2e-5)
# loss = {
#     'mlm_output': SparseCategoricalCrossentropy(from_logits=True),
#     'nsp_output': BinaryCrossentropy(from_logits=True)
# }
# metrics = {
#     'mlm_output': 'accuracy',
#     'nsp_output': 'accuracy'
# }



# def masked_sparse_categorical_crossentropy(y_true, y_pred):
#     # Create a mask to ignore `-1` in labels for loss calculation
#     mask = tf.cast(tf.not_equal(y_true, -1), tf.float32)
#     # Compute sparse categorical crossentropy loss
#     loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)
#     # Apply the mask
#     loss *= mask
#     # Calculate mean loss only over non-masked elements
#     return tf.reduce_sum(loss) / tf.reduce_sum(mask)

# # Ensure the custom loss function is used when compiling the model
# bert_model.compile(
#     optimizer=optimizer,
#     loss={'mlm_output': masked_sparse_categorical_crossentropy, 'nsp_output': BinaryCrossentropy(from_logits=True)},
#     metrics=metrics
# )
# # epochs = 3 
# # bert_model.fit(train_dataset, epochs=epochs)

In [108]:
# class SmallBERT(Model):
#     def __init__(self, vocab_size, num_layers, d_model, num_heads, dff, max_pos=512, rate=0.1):
#         super(SmallBERT, self).__init__()
#         self.token_embedding = Embedding(vocab_size, d_model)
#         self.position_embedding = Embedding(max_pos, d_model)
#         self.segment_embedding = Embedding(2, d_model)  # Only 2 segments assumed
        
#         self.enc_layers = [TransformerEncoderV3(num_layers=1, d_model=d_model, num_heads=num_heads, dff=dff,
#                                                 vocab_size=vocab_size, max_pos=max_pos) for _ in range(num_layers)]
        
#         self.dropout = Dropout(rate)
#         self.final_layer = Dense(vocab_size)  # Prediction layer for MLM
#         self.nsp_classifier = Dense(2, activation='softmax')  # NSP output

#     def call(self, input_ids, segment_ids, training=False):
#         seq_length = tf.shape(input_ids)[1]
#         position_ids = tf.range(seq_length)
        
#         x = self.token_embedding(input_ids) + self.position_embedding(position_ids) + self.segment_embedding(segment_ids)
#         x = self.dropout(x, training=training)
        
#         for encoder in self.enc_layers:
#             x = encoder(x)
        
#         logits = self.final_layer(x)
#         pooled_output = self.dropout(sequence_output[:, 0, :], training=training)  # Use the output of the [CLS] token
#         nsp_output = self.nsp_classifier(pooled_output)
#         return logits, nsp_output  

# vocab_size = 10000  # Smaller vocabulary size for simplicity
# num_layers = 2  # Fewer layers
# d_model = 128  # Smaller dimensionality
# num_heads = 4
# dff = 512
# small_bert = SmallBERT(vocab_size, num_layers, d_model, num_heads, dff)